In [1]:
import random
import sys
import io
import time
import pandas as pd
from itertools import combinations
from gurobipy import *
from clean_data_cor import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# rota1 = ['uno','dos','tres', 'cuatro', 'cinco']
# rota = [0] + rota1

# OD = [('uno','dos'),
#     ('uno','tres'),
#     ('uno','cuatro'),
#     ('uno','cinco'),
#     ('dos','tres'),
#     ('dos','cuatro'),
#     ('dos','cinco'),
#     ('tres','cuatro'),
#     ('tres','cinco'),
#     ('cuatro','cinco'),]

# AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
# NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

# I = ['uno','dos','tres', 'cuatro']
# I2 = [0] + I

# J = ['dos','tres','cuatro', 'cinco']
# V = ['p']
# T = [0]
# Q = 700 
# n = len(rota) - 1 
# VK = {'p':[1,5,7]}



# # [start] calcular la demanda 
# dd = {(i,j,v,k,t): np.random.randint(1,100) for i,j in OD for v in V for k in VK[v] for t in T }


# reve = {}
# for i,j in OD:
#     for v in V:
#         aleat = sorted(random.sample(range(50, 2000 + 1), len(VK[v])), reverse=True)
#         for k,valor in enumerate(aleat):
#             reve[(i,j,v,VK[v][k])] = valor


# demanda = pd.DataFrame.from_dict(dd, orient="index")
# demanda.reset_index(inplace=True)
# demanda[['i', 'j', 'v', 'k', 't']] = pd.DataFrame(demanda['index'].tolist(), index=demanda.index)
# demanda = demanda.drop("index", axis=1)
# demanda = demanda[['i', 'j', 'v', 'k', 't', 0]]
# demanda.columns = ['Origin','Destination','Vagon','Class','DBD','Bookings']


# preco_ = pd.DataFrame.from_dict(reve, orient="index")
# preco_.reset_index(inplace=True)
# preco_[['i', 'j', 'v', 'k']] = pd.DataFrame(preco_['index'].tolist(), index=preco_.index)
# preco_ = preco_.drop("index", axis=1)
# preco_ = preco_[['i', 'j', 'v', 'k', 0]]
# preco_.columns = ['Origin','Destination','Vagon','Class','Revenue']


# demanda.to_excel("/home/wilmer/Documentos/Codes/tesis/Instancias/test/demanda.xlsx", index=False)
# preco_.to_excel("/home/wilmer/Documentos/Codes/tesis/Instancias/test/preco.xlsx", index=False)

In [3]:
demanda = pd.read_excel("/home/wilmer/Documentos/Codes/tesis/Instancias/test/demanda.xlsx")
preco = pd.read_excel("/home/wilmer/Documentos/Codes/tesis/Instancias/test/preco.xlsx")
perio = 1
Q = 550
rota1 = ['uno','dos','tres', 'cuatro', 'cinco']

# rota0 = pd.read_csv("/home/wilmer/Documentos/Codes/tesis/Instancias/instancia3/rota.csv")
# rota1 = eval(rota0['Route'][0])

In [4]:
if perio != 0:
    periodo_lim = sorted(demanda['DBD'].unique().tolist())[:perio]
    print(periodo_lim)
    demanda = demanda[demanda['DBD'].isin(periodo_lim)]

rota = [0] + rota1

# Clean_Data
# find parameters
origin_cor = preco['Origin'].unique().tolist()
destin_cor = preco['Destination'].unique().tolist()

oridest = preco[['Origin','Destination']].apply(lambda x: (x['Origin'],x['Destination']), axis=1)
oridest = oridest.unique().tolist()

stations = origin_cor + [i for i in destin_cor if i not in origin_cor]

vagones = preco['Vagon'].unique().tolist()
clases = {v: sorted(preco[preco['Vagon']==v]['Class'].unique().tolist()) for v in vagones}

periodo = sorted(demanda['DBD'].unique().tolist(), reverse=True)    

# # [start] converting demand into behavioural
# demanda["DemandaComport"] = demanda.apply(lambda fila: clases[fila["Vagon"]][clases[fila["Vagon"]].index(fila["Class"]):][::-1] , axis=1)
# demanda["DemPotencialTot"] = demanda.apply(behav_demand, axis=1, df=demanda)
# demanda.columns = ['Origin', 'Destination', 'Vagon', 'Class', 'DBD', "Bookings1", 'PL', 'Bookings']
# # [end] converting demand into behavioural

# sort data revenue
preco = preco.sort_values(by=['Origin', 'Destination', 'Vagon', 'Revenue'], ascending=[True, True, True, False])
preco
# Transform to dictionary
dem_cor = demanda.set_index(['Origin', 'Destination', 'Vagon', 'Class', 'DBD'])['Bookings'].to_dict()
preco_cor = preco.set_index(['Origin', 'Destination', 'Vagon', 'Class'])['Revenue'].to_dict()

#todas las combinaciones de los indices
# indexCombiPre = [tuple(x) for x in preco[['Origin','Destination','Vagon','Class']].to_numpy()] #para el precio
indexCombiDem = [tuple(x) for x in demanda[['Origin','Destination','Vagon','Class','DBD']].to_numpy()] #para l ademanda
index_Cero = [(0, i, v, c, t) for i, v, t in product(stations, vagones, periodo) for c in clases[v]]
indexCombiDem0 = indexCombiDem + index_Cero

I, J, OD, V,  T, stations, VK, P, d, indexCombiDem, indexCombiDem0 = origin_cor, destin_cor, oridest, vagones, periodo, stations, clases, preco_cor, dem_cor, indexCombiDem, indexCombiDem0


[0]


In [5]:
AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

I = [i for i in rota if i in I]
I2 = [0] + I# tambem se toma para el para tudo da restriccion de fluxo
J = [i for i in rota if i in J]
n = len(rota)-1


BR = {}
for i,j in NAD:
    listTemp = list(combinations(rota1[rota1.index(i):rota1.index(j)+1], 2))
    listTemp =  {(ii,jj) for ii,jj in listTemp if (ii,jj) in OD and (ii, jj) != (i,j)} #rota1.index(jj) == rota1.index(ii)+1 and 
    BR[(i,j)] = listTemp

# # indices
# index0 = [(i,j,v,k,t) for i in I2 for j in J for v in V for k in VK[v] for t in T if rota.index(i) <  rota.index(j) ]#and (i,j) in OD]
# index1 = [(i,j,v,k,t) for i in I for j in J for v in V for k in VK[v] for t in T if rota.index(i) <  rota.index(j) ]#and (i,j) in OD]
index = [(i,j,v,k,t) for i,j,v,k,t in  indexCombiDem if (i,j) in BR.keys()]


In [6]:
# def create_model(I, I2, J, V, VK, T, OD, NAD, BR, P, Q, d, index0, index, index1, rota, indexCombiDem, indexCombiDem0):
model = Model("Modelo 1.1.1")

# variables de decicion
X = model.addVars(indexCombiDem0, vtype=GRB.INTEGER , name="X")
Y = model.addVars(indexCombiDem, vtype=GRB.INTEGER , name="Y")
A = model.addVars(I2, vtype=GRB.INTEGER , name="A")
BNA = model.addVars(index, vtype=GRB.BINARY , name="BNA")


# funcion objetivo
model.setObjective(
    quicksum(P[(i,j,v,k)]*X[(i,j,v,k,t)] for i,j,v,k,t in indexCombiDem),
    sense = GRB.MAXIMIZE
)

# restricciones
for i in I:

    # restricao .2
    model.addConstr(
        A[i] == A[I2[I2.index(i)-1]] - 
        quicksum(X[I2[I2.index(i)-1],j,v,k,t] for i_,j,v,k,t in indexCombiDem0 if (I2[I2.index(i)-1] == i_)) + 
        quicksum(X[i_,j,v,k,t] for i_,j,v,k,t in indexCombiDem0 if (j == i)),
        name=f"Dispo_{i}"
    )

    # restricao .3
    model.addConstr(
        quicksum((X[i,j,v,k,t]) for i_,j,v,k,t in indexCombiDem if i == i_) <= A[i],
        name=f"Cap_{i}"
    )

    # restricao .6
    model.addConstr(
        quicksum(Y[i,j,v,VK[v][0],t] for i_,j,v,k,t in indexCombiDem if (i == i_) and (VK[v][0] == k)) <= Q, 
        name=f"AuthoCap_{i}"
    )


for i,j,v,k,t in indexCombiDem:

    VK_ = demanda.loc[(demanda["Origin"]==i) & (demanda["Destination"]==j) & (demanda["Vagon"]==v) & (demanda["DBD"]==t)]["Class"].to_list()
    print(VK_)
    pos_k = VK_.index(k)
    last_k = VK_[-1]

    # restricao .5
    model.addConstr(
        X[i,j,v,k,t] <= d[i,j,v,k,t],
        name = f"Assig_({i},{j},{v},{k},{t})"
    )

    # restricao .demanda comportamental
    if pos_k >= 1:
        model.addConstr(
            quicksum(X[i,j,v,kk,t] for kk in VK_[0:pos_k+1]) <= d[i,j,v,k,t],
            name = f"DemComp({i},{j},{v},{k},{t})"
        )


    if k != last_k:
        
        # restricao .4
        model.addConstr(
            Y[i,j,v,k,t] >= Y[i,j,v,VK_[pos_k+1],t], 
            name=f"Classe_({i},{j},{v},{k},{t})"
        )

        # restricao .8
        model.addConstr(
            Y[i,j,v,k,t] >=  X[i,j,v,k,t] + Y[i,j,v,VK_[pos_k+1],t],
            name=f"Autho_({i},{j},{v},{k},{t})"
        )
    else:

        # restricao .7
        model.addConstr(
            Y[i,j,v,k,t] >=  X[i,j,v,k,t],
            name=f"Autho_({i},{j},{v},{k},{t})"
        )


    if (i,j) in NAD:
        # aqui el (i,j)=(o,d)

        # restricao .9.1
        model.addConstr(
            BNA[i,j,v,k,t] <= Y[i,j,v,k,t],
            # name = f"activ_bin_autho_low_({o},{d_},{v},{k},{t})"
        )
        
        # restricao .9.2
        model.addConstr(
            Y[i,j,v,k,t] <= Q*BNA[i,j,v,k,t],
            # name = f"activ_bin_autho_top_({o},{d_},{v},{k},{t})"
        )

        for ii,jj,vv,kk,tt in indexCombiDem:
            if (ii,jj) in BR[(i,j)] and v == vv and k == kk and t == tt:
                # restricao .10.1
                model.addConstr(
                    Y[ii,jj,vv,kk,tt] <= Q*BNA[i,j,vv,kk,tt],
                    # name = f"pru1({o},{d_},{i},{j},{v},{k},{t})"
                )

                # restricao .10.2
                model.addConstr(
                    BNA[i,j,vv,kk,tt] <= Y[ii,jj,vv,kk,tt],
                    # name = f"pru2({o},{d_},{i},{j},{v},{k},{t})"
                )


for i,j,v,k,t in indexCombiDem0:
    # restricao .11
    if i == 0:
        model.addConstr(
            X[i,j,v,k,t] == 0,
            name = f"Assig_({0},{j},{v},{k},{t})"
        )

# restricao .12
model.addConstr(
    A[0] == Q,
    name = f"Cap_0")

# Optimizar o modelo
model.optimize()



# model.write("/home/wilmer/Documentos/Codes/tesis/Instancias/test/modelo.lp")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-25
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
[1, 5]
[1, 5]
[1, 5, 7]
[1, 5, 7]
[1, 5, 7]
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 198 rows, 77 columns and 436 nonzeros
Model fingerprint: 0x31ac47f9
Variable types: 0 continuous, 77 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [6e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 6e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 198 rows and 77 columns
Presolve time: 0.00s
Presolve: All rows and 

In [7]:
"S29" in rota

False

In [8]:
demanda.loc[(demanda["Origin"]=="S10") & (demanda["Destination"]=="S24")]


,Origin,Destination,Vagon,Class,DBD,Bookings


In [9]:
demanda.DBD.unique()

array([0])